In [93]:
import pandas as pd
import numpy as np
import pickle

In [94]:
infile = open('../data/processed/scherzer_with_batters.pickle','rb')
pb = pickle.load(infile)
infile.close()

infile = open('../data/processed/X_train_trans.pickle','rb')
X_train = pickle.load(infile)
infile.close()

infile = open('../data/processed/y_train_trans.pickle','rb')
y_train = pickle.load(infile)
infile.close()

In [120]:
pb.batter_id.nunique()

192

In [122]:
200 * 180

36000

In [95]:
# '''
# "Sunny day went to ...." picnic? park?

# NLP problem is everywhere
# Ng work with shakespeare

# Our book is -> each sequence is at bat (pitcher vs. batter)

# 'Schzer threw a ff to batter ast the first pitch.  The next pitch xyz, the pitch after was '

# Murat suggests -> find LSTM tutorial with text guessing -> 
# will do same thing but with pitches -> 
# their data with pitch data (skip tokenization etc.)

# Look at how they pass the data

# Keras will pad null pitches for an at bat

# Each row is an at bat -> need to figure out what the numbers will be

# Each row is an at bat, padded by max length, sequence is pitch type.

# '''

In [96]:
pb.columns

Index(['pitch_type', 'game_date_x', 'sv_id', 'batter_id', 'pitch_number',
       'release_speed', 'zone', 'stand', 'home_team', 'on_3b', 'on_2b',
       'on_1b', 'outs_when_up', 'inning', 'release_spin_rate', 'opp_score',
       'nats_score', 'if_fielding_alignment', 'of_fielding_alignment',
       'nats_home1_away0', 'balls_strikes', 'all_runners', 'pitch_season',
       'pitch_game', 'pitch_bat_gm', 'game_date_y', 'shift_date',
       'player_name', 'total_pitches', 'hits', 'abs', 'whiffs', 'swings',
       'takes', 'k', 'walk', 'single', 'double', 'triple', 'hr', 'line_drive',
       'ground_ball', 'fly_ball', 'popup', 'rbi', 'sac', 'ba', 'slg', 'iso',
       'babip'],
      dtype='object')

In [97]:
pitch_vals = pb.pitch_type.unique().tolist()

pitch_vals = dict(zip(pitch_vals, range(1, len(pitch_vals) + 1)))

In [98]:
# pb_df = pb[['pitch_type', 'pitch_number', 'pitch_bat_gm']][:21]
# pb_df;

In [99]:
pb.loc[(pb.pitch_number == 0), 'new_ab'] = 'yes'
pb.new_ab = pb.new_ab.fillna(value='no')

pb['pitch_val'] = pb.pitch_type.apply(lambda p: pitch_vals[p])
# pb_df = pb_df[['pitch_type', 'pitch_val', 'pitch_number', 'pitch_bat_gm', 'new_ab']]

pb

,pitch_type,game_date_x,sv_id,batter_id,pitch_number,release_speed,zone,stand,home_team,on_3b,...,fly_ball,popup,rbi,sac,ba,slg,iso,babip,new_ab,pitch_val
0,FF,2019-03-28,190328_170717,607043,0,93.7,6.0,L,WSH,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,yes,1
1,FF,2019-03-28,190328_170732,607043,1,94.2,5.0,L,WSH,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,no,1
2,FF,2019-03-28,190328_170752,607043,2,96.3,5.0,L,WSH,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,no,1
3,SL,2019-03-28,190328_170825,624413,0,85.6,6.0,R,WSH,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,yes,2
4,FF,2019-03-28,190328_170842,624413,1,95.5,12.0,R,WSH,0,...,0,0,0,0,0.000000,0.000000,0.000000,0.000000,no,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2765,CU,2019-09-24,190925_005705,664068,1,79.1,4.0,R,WSH,0,...,76,27,36,3,0.259009,0.481982,0.222973,0.334495,no,5
2766,SL,2019-09-24,190925_005726,664068,2,84.8,14.0,R,WSH,0,...,76,27,36,3,0.259009,0.481982,0.222973,0.334495,no,2
2767,FF,2019-09-24,190925_005750,664068,3,97.4,8.0,R,WSH,0,...,76,27,36,3,0.259009,0.481982,0.222973,0.334495,no,1
2768,CH,2019-09-24,190925_005823,664068,4,84.4,9.0,R,WSH,0,...,76,27,36,3,0.259009,0.481982,0.222973,0.334495,no,4


In [101]:
ab_list = []
pitchs_ab = []

for ab in range(0, pb.shape[0]):
    ab_val = pb.at[ab, 'new_ab']
    
# Case for last pitch of the season is a one pitch at bat
    if ab_val == 'yes' and ab == pb.shape[0] - 1:
        pitchs_ab = []
        pitchs_ab.append(pb.at[ab, 'pitch_val'])
        ab_list.append(pitchs_ab)
        
# Case for last pitch of the season is not a one pitch at bat
    elif ab_val == 'no' and ab == pb.shape[0] - 1:
        pitchs_ab.append(pb.at[ab, 'pitch_val'])
        ab_list.append(pitchs_ab)
        
# Case for a one pitch at bat that is not the last of the season
    elif ab_val == 'yes' and ab == pb.at[ab + 1, 'new_ab'] == 'yes':
        pitchs_ab = []
        pitchs_ab.append(pb.at[ab, 'pitch_val'])
        ab_list.append(pitchs_ab)
        
# Case for first pitch of an at bat
    elif ab_val == 'yes' and pb.at[ab + 1, 'new_ab'] == 'no':
        pitchs_ab = []
        pitchs_ab.append(pb.at[ab, 'pitch_val'])

# Case for pitch that is last pitch of an at bat
    elif ab_val == 'no' and pb.at[ab + 1, 'new_ab'] == 'yes':
        pitchs_ab.append(pb.at[ab, 'pitch_val'])
        ab_list.append(pitchs_ab)

# Case for pitch that is neither the first nor last pitch in an at bat
    else:
        pitchs_ab.append(pb.at[ab, 'pitch_val'])

ab_list

[[1, 1, 1],
 [2, 1, 1, 3, 1, 4, 3],
 [1, 4, 4],
 [5, 1, 1, 4],
 [1, 2, 1, 4, 5],
 [2, 1, 2, 1],
 [1, 1, 1, 4],
 [4, 1, 1],
 [1, 5, 1, 1, 4, 3],
 [2, 1, 1, 1, 4],
 [1, 1, 3],
 [4, 1, 5, 1, 1, 4, 1],
 [1, 2],
 [1, 1],
 [2, 4, 2, 2, 4, 2],
 [1, 4, 1],
 [4, 4, 1],
 [1, 2, 1],
 [1, 5, 3],
 [4, 4, 4, 1, 5, 3, 1],
 [2, 4],
 [1, 1, 4, 4, 1, 5],
 [1, 4],
 [1, 2, 2, 2, 1],
 [5, 1, 3, 4, 1, 1, 4],
 [1, 5, 4, 4],
 [1, 1],
 [2, 1],
 [1, 3, 1, 3, 4, 4],
 [1, 2, 1, 1, 1, 1, 2],
 [2, 2, 2],
 [1, 3, 1, 4, 1],
 [1, 1, 5, 1, 1],
 [2, 1, 1, 5, 1, 4],
 [1, 2, 2],
 [2, 1, 5],
 [5, 2, 2],
 [3, 1, 1, 1, 1, 4, 3],
 [1, 4, 2, 1, 4, 1],
 [1, 1, 4, 1],
 [1, 5, 1, 5, 5],
 [1, 4],
 [1, 2, 1, 1, 2, 2, 1],
 [1, 2, 1, 4],
 [2, 1, 4],
 [4, 3, 1, 1, 5],
 [4, 2, 4],
 [5, 2, 2, 4, 1],
 [1, 1, 1, 3],
 [1, 1, 1, 4, 4, 1],
 [1, 1, 4, 3, 1, 1],
 [1, 1, 1, 4, 4],
 [1, 1, 2, 1, 2, 2],
 [1, 2, 2, 2],
 [1, 1, 2, 1],
 [1, 1],
 [1, 5, 4, 3, 4, 1],
 [2, 1, 1, 1],
 [2, 1, 1],
 [5, 5, 3, 4, 1, 1, 4],
 [1, 1, 3],
 [2, 2, 1, 1],
 [1, 2,

In [171]:
data = pb.pitch_val.tolist()

In [178]:
len(data)
val_data = data[2000:2770]
val_data;

In [129]:
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np

In [180]:
# univariate one step problem with lstm
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.preprocessing.sequence import TimeseriesGenerator

# define dataset
series = array(data)
series_val = array(val_data)


# reshape to [10, 1]
n_features = 1
series = series.reshape((len(series), n_features))
series_val = series_val.reshape((len(series_val), n_features))

# define generator
n_input = 3
generator = TimeseriesGenerator(series, series, length=n_input, batch_size=8)
gen_val = TimeseriesGenerator(series_val, series_val, length=n_input, batch_size=8)

# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit_generator(generator, steps_per_epoch=1, epochs=500, verbose=1, validation_data=gen_val)
# # make a one step prediction out of sample
# x_input = array([9, 10]).reshape((1, n_input, n_features))
# yhat = model.predict(x_input, verbose=0)
# print(yhat)


Epoch 1/500
1/1 [==============================] - 1s 1s/step - loss: 6.5558 - val_loss: 7.7575
Epoch 2/500
1/1 [==============================] - 0s 197ms/step - loss: 6.3922 - val_loss: 7.6140
Epoch 3/500
1/1 [==============================] - 0s 216ms/step - loss: 5.8271 - val_loss: 7.4721
Epoch 4/500
1/1 [==============================] - 0s 204ms/step - loss: 7.0809 - val_loss: 7.3304
Epoch 5/500
1/1 [==============================] - 0s 201ms/step - loss: 1.5529 - val_loss: 7.2000
Epoch 6/500
1/1 [==============================] - 0s 200ms/step - loss: 7.5145 - val_loss: 7.0677
Epoch 7/500
1/1 [==============================] - 0s 208ms/step - loss: 7.5407 - val_loss: 6.9342
Epoch 8/500
1/1 [==============================] - 0s 204ms/step - loss: 9.1825 - val_loss: 6.7982
Epoch 9/500
1/1 [==============================] - 0s 202ms/step - loss: 8.0136 - val_loss: 6.6591
Epoch 10/500
1/1 [==============================] - 0s 299ms/step - loss: 6.7977 - val_loss: 6.5195
Epoch 11/500

1/1 [==============================] - 0s 189ms/step - loss: 2.4257 - val_loss: 1.5059
Epoch 84/500
1/1 [==============================] - 0s 211ms/step - loss: 2.2907 - val_loss: 1.5064
Epoch 85/500
1/1 [==============================] - 0s 281ms/step - loss: 2.2346 - val_loss: 1.5057
Epoch 86/500
1/1 [==============================] - 0s 198ms/step - loss: 2.1603 - val_loss: 1.5087
Epoch 87/500
1/1 [==============================] - 0s 210ms/step - loss: 2.2250 - val_loss: 1.5126
Epoch 88/500
1/1 [==============================] - 0s 282ms/step - loss: 2.0237 - val_loss: 1.5103
Epoch 89/500
1/1 [==============================] - 0s 282ms/step - loss: 2.6541 - val_loss: 1.5198
Epoch 90/500
1/1 [==============================] - 0s 263ms/step - loss: 0.6195 - val_loss: 1.5280
Epoch 91/500
1/1 [==============================] - 0s 249ms/step - loss: 1.1776 - val_loss: 1.5338
Epoch 92/500
1/1 [==============================] - 0s 203ms/step - loss: 2.1787 - val_loss: 1.5268
Epoch 93/500


Epoch 165/500
1/1 [==============================] - 0s 197ms/step - loss: 2.1897 - val_loss: 1.8234
Epoch 166/500
1/1 [==============================] - 0s 248ms/step - loss: 1.0698 - val_loss: 1.8181
Epoch 167/500
1/1 [==============================] - 0s 228ms/step - loss: 3.2449 - val_loss: 1.8083
Epoch 168/500
1/1 [==============================] - 0s 201ms/step - loss: 1.7411 - val_loss: 1.8098
Epoch 169/500
1/1 [==============================] - 0s 205ms/step - loss: 2.1722 - val_loss: 1.8128
Epoch 170/500
1/1 [==============================] - 0s 265ms/step - loss: 3.8892 - val_loss: 1.8136
Epoch 171/500
1/1 [==============================] - 0s 239ms/step - loss: 0.8366 - val_loss: 1.8167
Epoch 172/500
1/1 [==============================] - 0s 249ms/step - loss: 2.9315 - val_loss: 1.8094
Epoch 173/500
1/1 [==============================] - 0s 191ms/step - loss: 0.2844 - val_loss: 1.8050
Epoch 174/500
1/1 [==============================] - 0s 201ms/step - loss: 2.7875 - val_los

1/1 [==============================] - 0s 204ms/step - loss: 3.3006 - val_loss: 1.7947
Epoch 247/500
1/1 [==============================] - 0s 201ms/step - loss: 0.5339 - val_loss: 1.7924
Epoch 248/500
1/1 [==============================] - 0s 201ms/step - loss: 2.0934 - val_loss: 1.7911
Epoch 249/500
1/1 [==============================] - 0s 219ms/step - loss: 2.3009 - val_loss: 1.7897
Epoch 250/500
1/1 [==============================] - 0s 200ms/step - loss: 2.1789 - val_loss: 1.7860
Epoch 251/500
1/1 [==============================] - 0s 194ms/step - loss: 2.2653 - val_loss: 1.7839
Epoch 252/500
1/1 [==============================] - 0s 198ms/step - loss: 2.3677 - val_loss: 1.7836
Epoch 253/500
1/1 [==============================] - 0s 201ms/step - loss: 2.8808 - val_loss: 1.7820
Epoch 254/500
1/1 [==============================] - 0s 204ms/step - loss: 2.0971 - val_loss: 1.7817
Epoch 255/500
1/1 [==============================] - 0s 212ms/step - loss: 0.8228 - val_loss: 1.7817
Epoc

Epoch 328/500
1/1 [==============================] - 0s 237ms/step - loss: 2.0381 - val_loss: 1.9149
Epoch 329/500
1/1 [==============================] - 0s 197ms/step - loss: 2.2934 - val_loss: 1.9057
Epoch 330/500
1/1 [==============================] - 0s 220ms/step - loss: 2.8154 - val_loss: 1.8940
Epoch 331/500
1/1 [==============================] - 0s 246ms/step - loss: 3.6382 - val_loss: 1.8882
Epoch 332/500
1/1 [==============================] - 0s 217ms/step - loss: 3.1089 - val_loss: 1.8830
Epoch 333/500
1/1 [==============================] - 0s 207ms/step - loss: 1.5851 - val_loss: 1.8809
Epoch 334/500
1/1 [==============================] - 0s 273ms/step - loss: 1.9237 - val_loss: 1.8768
Epoch 335/500
1/1 [==============================] - 0s 216ms/step - loss: 0.8176 - val_loss: 1.8770
Epoch 336/500
1/1 [==============================] - 0s 199ms/step - loss: 1.9341 - val_loss: 1.8764
Epoch 337/500
1/1 [==============================] - 0s 202ms/step - loss: 0.9430 - val_los

Epoch 410/500
1/1 [==============================] - 0s 205ms/step - loss: 1.2892 - val_loss: 1.9825
Epoch 411/500
1/1 [==============================] - 0s 196ms/step - loss: 1.8558 - val_loss: 1.9733
Epoch 412/500
1/1 [==============================] - 0s 211ms/step - loss: 1.0973 - val_loss: 1.9711
Epoch 413/500
1/1 [==============================] - 0s 209ms/step - loss: 2.3811 - val_loss: 1.9612
Epoch 414/500
1/1 [==============================] - 0s 200ms/step - loss: 1.5061 - val_loss: 1.9550
Epoch 415/500
1/1 [==============================] - 0s 194ms/step - loss: 2.3175 - val_loss: 1.9528
Epoch 416/500
1/1 [==============================] - 0s 204ms/step - loss: 3.0436 - val_loss: 1.9504
Epoch 417/500
1/1 [==============================] - 0s 210ms/step - loss: 2.1835 - val_loss: 1.9485
Epoch 418/500
1/1 [==============================] - 0s 207ms/step - loss: 2.2964 - val_loss: 1.9414
Epoch 419/500
1/1 [==============================] - 0s 191ms/step - loss: 1.4660 - val_los

1/1 [==============================] - 0s 198ms/step - loss: 3.3030 - val_loss: 1.9239
Epoch 492/500
1/1 [==============================] - 0s 203ms/step - loss: 1.8935 - val_loss: 1.9454
Epoch 493/500
1/1 [==============================] - 0s 206ms/step - loss: 1.5022 - val_loss: 1.9753
Epoch 494/500
1/1 [==============================] - 0s 202ms/step - loss: 1.5105 - val_loss: 2.0143
Epoch 495/500
1/1 [==============================] - 0s 198ms/step - loss: 0.8798 - val_loss: 2.0665
Epoch 496/500
1/1 [==============================] - 0s 209ms/step - loss: 1.2473 - val_loss: 2.1280
Epoch 497/500
1/1 [==============================] - 0s 201ms/step - loss: 3.5646 - val_loss: 2.1798
Epoch 498/500
1/1 [==============================] - 0s 202ms/step - loss: 1.0660 - val_loss: 2.2301
Epoch 499/500
1/1 [==============================] - 0s 195ms/step - loss: 1.8066 - val_loss: 2.2826
Epoch 500/500
1/1 [==============================] - 0s 195ms/step - loss: 2.2617 - val_loss: 2.3309


In [185]:
x_input = array([2, 1, 1]).reshape((1, n_input, n_features))
yhat = model.predict(batch_0[0], verbose=1)
print(yhat)

3/3 [==============================] - 0s 412us/step
[[1.7818307]
 [1.7904371]
 [1.8187258]]


In [186]:
batch_0[1]

array([[[2]],

       [[1]],

       [[1]]])

In [ ]:
'''
Figure out y-val OHE

SEQUENCE_LENGTH = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i: i + SEQUENCE_LENGTH])
    next_chars.append(text[i + SEQUENCE_LENGTH])
print(f'num training examples: {len(sentences)}')


text is data
'''

In [130]:

data = np.array([[i] for i in data])
targets = np.array([[i] for i in data])
data_gen = TimeseriesGenerator(data, targets,
                               length=3, sampling_rate=1,
                               batch_size=3)
# assert len(data_gen) == 20
# batch_0 = data_gen[0]
# x, y = batch_0
# assert np.array_equal(x,
#                       np.array([[[0], [2], [4], [6], [8]],
#                                 [[1], [3], [5], [7], [9]]]))
# assert np.array_equal(y,
#                       np.array([[10], [11]]))


In [132]:
len(data_gen)

923

In [147]:
# batch_0 = data_gen[0]
# x, y = batch_0
# # x[0, :, :], y.shape

# batch_1 = data_gen[1]
# x_1, y_1 = batch_1
# x_1, y_1

In [167]:
model = Sequential()
model.add(LSTM(128, input_shape = (3, 1)))
model.add(Dense(1, activation='sigmoid'))
# model.add(Activation('softmax'))

In [168]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [169]:
model.summary()

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_21 (LSTM)               (None, 128)               66560     
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 129       
Total params: 66,689
Trainable params: 66,689
Non-trainable params: 0
_________________________________________________________________


In [170]:
model.fit_generator(data_gen, steps_per_epoch=1, epochs=20, shuffle=False)

# model.fit_generator(generator, steps_per_epoch=1, epochs=200, verbose=0)

Epoch 1/20


ValueError: Error when checking target: expected dense_13 to have 2 dimensions, but got array with shape (3, 1, 1)

In [55]:
at_bats = pd.DataFrame(data = ab_list)
at_bats = at_bats.fillna(value=0)
at_bats.shape[1] - 1
cols = ['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12']
at_bats.columns = cols
at_bats = at_bats.applymap(lambda v: int(v))
at_bats

,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12
0,1,1,1,0,0,0,0,0,0,0,0,0
1,2,1,1,3,1,4,3,0,0,0,0,0
2,1,4,4,0,0,0,0,0,0,0,0,0
3,5,1,1,4,0,0,0,0,0,0,0,0
4,1,2,1,4,5,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
629,4,4,1,3,0,0,0,0,0,0,0,0
630,2,2,4,1,4,2,0,0,0,0,0,0
631,1,5,3,0,0,0,0,0,0,0,0,0
632,1,5,2,1,4,0,0,0,0,0,0,0


In [92]:
flat_list = []
for i in range(0, at_bats.shape[0]):
#     print(i)
    flat_list.append(at_bats.iloc[0, :].tolist())


flat_list = [item for sublist in ab_list for item in sublist]
flat_list
# at_bats.iloc[0, :].tolist()

[1,
 1,
 1,
 2,
 1,
 1,
 3,
 1,
 4,
 3,
 1,
 4,
 4,
 5,
 1,
 1,
 4,
 1,
 2,
 1,
 4,
 5,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 1,
 5,
 1,
 1,
 4,
 3,
 2,
 1,
 1,
 1,
 4,
 1,
 1,
 3,
 4,
 1,
 5,
 1,
 1,
 4,
 1,
 1,
 2,
 1,
 1,
 2,
 4,
 2,
 2,
 4,
 2,
 1,
 4,
 1,
 4,
 4,
 1,
 1,
 2,
 1,
 1,
 5,
 3,
 4,
 4,
 4,
 1,
 5,
 3,
 1,
 2,
 4,
 1,
 1,
 4,
 4,
 1,
 5,
 1,
 4,
 1,
 2,
 2,
 2,
 1,
 5,
 1,
 3,
 4,
 1,
 1,
 4,
 1,
 5,
 4,
 4,
 1,
 1,
 2,
 1,
 1,
 3,
 1,
 3,
 4,
 4,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 3,
 1,
 4,
 1,
 1,
 1,
 5,
 1,
 1,
 2,
 1,
 1,
 5,
 1,
 4,
 1,
 2,
 2,
 2,
 1,
 5,
 5,
 2,
 2,
 3,
 1,
 1,
 1,
 1,
 4,
 3,
 1,
 4,
 2,
 1,
 4,
 1,
 1,
 1,
 4,
 1,
 1,
 5,
 1,
 5,
 5,
 1,
 4,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 4,
 2,
 1,
 4,
 4,
 3,
 1,
 1,
 5,
 4,
 2,
 4,
 5,
 2,
 2,
 4,
 1,
 1,
 1,
 1,
 3,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 1,
 4,
 3,
 1,
 1,
 1,
 1,
 1,
 4,
 4,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 5,
 4,
 3,
 4,
 1,
 2,
 1,


In [12]:
# # prepare the dataset of input to output pairs encoded as integers
# raw_text = flat_list
# n_chars = len(raw_text)

# seq_length = 1
# dataX = []
# dataY = []
# for i in range(0, n_chars - seq_length):
#     seq_in = raw_text[i:i + seq_length]
#     seq_out = raw_text[i + seq_length]
#     dataX.append([char_to_int[char] for char in seq_in])
#     dataY.append(char_to_int[seq_out])
# n_patterns = len(dataX)
# print("Total Patterns: ", n_patterns)

In [57]:
flat_list = [item for sublist in ab_list for item in sublist]
flat_list;

In [58]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [15]:
# # load ascii text and covert to lowercase
# filename = 'wonderland.txt'
# raw_text = open(filename, 'r', encoding='utf-8').read()
# raw_text = raw_text.lower()

In [16]:
# # create mapping of unique chars to integers, and a reverse mapping
# chars = sorted(list(set(raw_text)))
# char_to_int = dict((c, i) for i, c in enumerate(chars))
# int_to_char = dict((i, c) for i, c in enumerate(chars))

In [17]:
# # summarize the loaded data
# n_chars = len(raw_text)
# n_vocab = len(chars)
# print("Total Characters: ", n_chars)
# print("Total Vocab: ", n_vocab)

In [18]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 11
# dataX = []
# dataY = []
# for i in range(0, n_chars - seq_length, 1):
#     seq_in = raw_text[i:i + seq_length]
#     seq_out = raw_text[i + seq_length]
#     dataX.append([char_to_int[char] for char in seq_in])
#     dataY.append(char_to_int[seq_out])
# n_patterns = len(dataX)
# print("Total Patterns: ", n_patterns)

## Jam my data into this tutorial
dataX = np.array(at_bats[['p1', 'p2', 'p3', 'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11']])
dataY = at_bats.pitch_val

n_patterns = len(dataX)

AttributeError: 'DataFrame' object has no attribute 'pitch_val'

In [19]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

X.shape, len(X), len(X[0])

NameError: name 'n_patterns' is not defined

In [20]:
# normalize
# X = X / float(n_vocab)



In [21]:
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

y = dataY

ValueError: zero-size array to reduction operation maximum which has no identity

In [22]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

NameError: name 'X' is not defined

In [23]:
# load the network weights
# filename = "weights-improvement-47-1.2219-bigger.hdf5"
# model.load_weights(filename)
# model.compile(loss='categorical_crossentropy', optimizer='adam')

In [24]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

Seed:


NameError: name 'int_to_char' is not defined

In [25]:
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

NameError: name 'n_vocab' is not defined

## Tutorial code

In [33]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()


# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))


# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)


# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)


# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))


# normalize
X = X / float(n_vocab)


# one hot encode the output variable
y = np_utils.to_categorical(dataY)


# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=3, batch_size=128, callbacks=callbacks_list)

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

## load the network weights
filename = "weights-improvement-47-1.2219-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam', workers = -1)


# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")


# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Total Characters:  163780
Total Vocab:  58
Total Patterns:  163680
Epoch 1/3
163680/163680 [==============================] - 549s 3ms/step - loss: 2.8930

Epoch 00001: loss improved from inf to 2.89304, saving model to weights-improvement-01-2.8930.hdf5
Epoch 2/3
163680/163680 [==============================] - 544s 3ms/step - loss: 2.6197

Epoch 00002: loss improved from 2.89304 to 2.61969, saving model to weights-improvement-02-2.6197.hdf5
Epoch 3/3
163680/163680 [==============================] - 534s 3ms/step - loss: 2.4504

Epoch 00003: loss improved from 2.61969 to 2.45041, saving model to weights-improvement-03-2.4504.hdf5


OSError: Unable to open file (unable to open file: name = 'weights-improvement-47-1.2219-bigger.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Hack tutorial code

In [38]:
for i in range(0, 10):
    seq_in = raw_text[i: i + seq_length]
    seq_out = raw_text[i + seq_length]
    print(seq_in)
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

project gutenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of
roject gutenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of 
oject gutenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of a
ject gutenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of an
ect gutenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of any
ct gutenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of anyo
t gutenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of anyon
 gutenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of anyone
gutenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of anyone 
utenberg's alice's adventures in wonderland, by lewis carroll

this ebook is for the use of

In [ ]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


# # load ascii text and covert to lowercase
# filename = "wonderland.txt"
# raw_text = open(filename, 'r', encoding='utf-8').read()
# raw_text = raw_text.lower()


# # create mapping of unique chars to integers, and a reverse mapping
# chars = sorted(list(set(raw_text)))
# char_to_int = dict((c, i) for i, c in enumerate(chars))
# int_to_char = dict((i, c) for i, c in enumerate(chars))


# # summarize the loaded data
# n_chars = len(raw_text)
# n_vocab = len(chars)
# print("Total Characters: ", n_chars)
# print("Total Vocab: ", n_vocab)


# prepare the dataset of input to output pairs encoded as integers
seq_length = 11
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i: i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)


# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))


# normalize
X = X / float(n_vocab)


# one hot encode the output variable
y = np_utils.to_categorical(dataY)


# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X, y, epochs=3, batch_size=128, callbacks=callbacks_list)

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

## load the network weights
filename = "weights-improvement-47-1.2219-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam', workers = -1)


# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")


# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

## Fresh start

In [108]:
from keras.preprocessing.sequence import pad_sequences

# pad sequence
padded = pad_sequences(ab_list, padding='post')
padded[1]

# Max pitches as rows, types of pitches

array([2, 1, 1, 3, 1, 4, 3, 0, 0, 0, 0, 0], dtype=int32)

In [114]:
padded[:5]

array([[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [2, 1, 1, 3, 1, 4, 3, 0, 0, 0, 0, 0],
       [1, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [5, 1, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 2, 1, 4, 5, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [110]:
# X_train_shape = numpy.reshape(X_train, 
#                               (X_train[0], X_train[1], 1))

In [90]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.fit(X_train, y_train, epochs=3, batch_size=128, callbacks=callbacks_list)

IndexError: tuple index out of range

In [105]:
X_train.columns

Index(['outs_when_up', 'inning', 'pitch_number', 'opp_score', 'nats_score',
       'nats_home1_away0', 'pitch_season', 'pitch_game', 'pitch_bat_gm',
       'total_pitches', 'abs', 'whiffs', 'swings', 'takes', 'k', 'walk',
       'single', 'double', 'triple', 'hr', 'line_drive', 'ground_ball',
       'fly_ball', 'popup', 'rbi', 'sac', 'ba', 'slg', 'iso', 'babip',
       'stand_r1', 'if_standard', 'if_strategic', 'of_strategic', '0_1', '0_2',
       '1_0', '1_1', '1_2', '2_0', '2_1', '2_2', '3_0', '3_1', '3_2',
       'fb:0_sb:0_tb:1', 'fb:0_sb:1_tb:0', 'fb:0_sb:1_tb:1', 'fb:1_sb:0_tb:0',
       'fb:1_sb:0_tb:1', 'fb:1_sb:1_tb:0', 'fb:1_sb:1_tb:1'],
      dtype='object')

In [106]:
# Add ba, slg as additional columns for example

# Take look at gated recurrent units, better with less data

In [107]:
X_train.shape[0]

1939

## Fresh start 2

from https://medium.com/@curiousily/making-a-predictive-keyboard-using-recurrent-neural-networks-tensorflow-for-hackers-part-v-3f238d824218

In [188]:
len(data)

2770

In [189]:
text = data

SEQUENCE_LENGTH = 3 #make pitches OHE, add five seasons, try gated recurrent networks, mind the input layer value
# Add innings as OHE columns or use ordinal values, number of pitches for batter, 
step = 1
sentences = []
next_chars = []
for i in range(0, len(text) - SEQUENCE_LENGTH, step):
    sentences.append(text[i: i + SEQUENCE_LENGTH])
    next_chars.append(text[i + SEQUENCE_LENGTH])
print(f'num training examples: {len(sentences)}')



num training examples: 2767


In [190]:
sentences[:5]

[[1, 1, 1], [1, 1, 2], [1, 2, 1], [2, 1, 1], [1, 1, 3]]

In [191]:
next_chars[:5]

[2, 1, 1, 3, 1]

In [202]:
##### char_indicies

X = np.zeros((len(sentences), SEQUENCE_LENGTH, 6), dtype=np.bool)
y = np.zeros((len(sentences), 6), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char] = 1
    y[i, next_chars[i]] = 1

In [203]:
X.shape

(2767, 3, 6)

In [206]:
X[2]

array([[False,  True, False, False, False, False],
       [False, False,  True, False, False, False],
       [False,  True, False, False, False, False]])

In [207]:
y[2]

array([False,  True, False, False, False, False])

In [200]:
np.unique(next_chars, return_counts=True)

(array([1, 2, 3, 4, 5]), array([1338,  573,  214,  401,  241]))

In [224]:
1338/len(next_chars)

0.4835561980484279

In [211]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQUENCE_LENGTH, 6)))
model.add(Dense(6, activation='softmax'))
# model.add(Activation('softmax'))

In [215]:
# optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X, y, validation_split=0.1, batch_size=32, epochs=20, shuffle=True).history

Train on 2490 samples, validate on 277 samples
Epoch 1/20
2490/2490 [==============================] - 1s 497us/step - loss: 1.4938 - accuracy: 0.4707 - val_loss: 1.4245 - val_accuracy: 0.4368
Epoch 2/20
2490/2490 [==============================] - 0s 137us/step - loss: 1.3452 - accuracy: 0.4888 - val_loss: 1.4198 - val_accuracy: 0.4368
Epoch 3/20
2490/2490 [==============================] - 0s 152us/step - loss: 1.3296 - accuracy: 0.4888 - val_loss: 1.3895 - val_accuracy: 0.4368
Epoch 4/20
2490/2490 [==============================] - 0s 140us/step - loss: 1.3264 - accuracy: 0.4888 - val_loss: 1.4016 - val_accuracy: 0.4368
Epoch 5/20
2490/2490 [==============================] - 0s 138us/step - loss: 1.3214 - accuracy: 0.4888 - val_loss: 1.3791 - val_accuracy: 0.4368
Epoch 6/20
2490/2490 [==============================] - 0s 133us/step - loss: 1.3199 - accuracy: 0.4888 - val_loss: 1.3941 - val_accuracy: 0.4368
Epoch 7/20
2490/2490 [==============================] - 0s 133us/step - loss:

In [222]:
model.predict(X[1:5, :, :])

array([[2.30174584e-04, 5.27137518e-01, 3.37407291e-01, 2.21930947e-02,
        7.38542378e-02, 3.91776524e-02],
       [1.93668311e-04, 4.83068764e-01, 2.77090818e-01, 2.86542159e-02,
        1.21618517e-01, 8.93739536e-02],
       [2.58301268e-04, 4.48310196e-01, 2.94483989e-01, 3.82069983e-02,
        1.05401002e-01, 1.13339536e-01],
       [2.06669050e-04, 5.10605752e-01, 1.16866253e-01, 9.06104296e-02,
        2.09130138e-01, 7.25807846e-02]], dtype=float32)

In [223]:
y[1:5]

array([[False,  True, False, False, False, False],
       [False,  True, False, False, False, False],
       [False, False, False,  True, False, False],
       [False,  True, False, False, False, False]])